Alameda County census tracts

ftp://ftp2.census.gov/geo/tiger/TIGER2010/TRACT/2010/tl_2010_06001_tract10.zip

I've produced a

    /Users/raymondyee/Downloads/tl_2010_06001_tract10/tl_2010_06001_tract10_4ps.topo.json
    
Let me link to that file in ./files

In [1]:
# print out the version of IPython used
import IPython
IPython.version_info

(4, 0, 0, '')

In [2]:
%%javascript
// https://github.com/mbostock/d3/issues/1693
require.config({
  paths: {
    d3: "http://d3js.org/d3.v3.min",
    queue: "http://d3js.org/queue.v1.min",
    topojson: "http://d3js.org/topojson.v1.min"
  }
});

require(["d3", "queue", "topojson"], function(d3, queue, topojson) {
  console.log(d3.version);
  console.log(queue.version);
  console.log(topojson.version);
});

<IPython.core.display.Javascript object>

In [3]:
%%html
<style type="text/css">

.counties {
  fill: none;
}

.states {
  fill: none;
  stroke: #fff;
  stroke-linejoin: round;
}

.q0-9 { fill:rgb(247,251,255); }
.q1-9 { fill:rgb(222,235,247); }
.q2-9 { fill:rgb(198,219,239); }
.q3-9 { fill:rgb(158,202,225); }
.q4-9 { fill:rgb(107,174,214); }
.q5-9 { fill:rgb(66,146,198); }
.q6-9 { fill:rgb(33,113,181); }
.q7-9 { fill:rgb(8,81,156); }
.q8-9 { fill:rgb(8,48,107); }

</style>

In [4]:
%%html
<div id="county_map" style="height:600px; width:100%"></div>
<script>
// https://github.com/mbostock/d3/issues/1693
require.config({
  paths: {
    d3: "http://d3js.org/d3.v3.min",
    queue: "http://d3js.org/queue.v1.min",
    topojson: "http://d3js.org/topojson.v1.min"
  }
});

require(["d3", "queue", "topojson"], function(d3, queue, topojson) {
  console.log(d3.version);
  console.log(queue.version);
  console.log(topojson.version);


    var width = 960,
        height = 500;

    var rateById = d3.map();

    var quantize = d3.scale.quantize()
        .domain([0, .15])
        .range(d3.range(9).map(function(i) { return "q" + i + "-9"; }));

    var path = d3.geo.path();

    var svg = d3.select('#county_map').append("svg")
        .attr("width", width)
        .attr("height", height);

    queue()
        .defer(d3.json, "files/data/us.json")
        .defer(d3.tsv, "files/temp/unemployment.tsv", function(d) { rateById.set(d.id, +d.rate); })
        .await(ready);

    function ready(error, us) {
      svg.append("g")
          .attr("class", "counties")
        .selectAll("path")
          .data(topojson.feature(us, us.objects.counties).features)
        .enter().append("path")
          .attr("class", function(d) { return quantize(rateById.get(d.id)); })
          .attr("d", path);

      svg.append("path")
          .datum(topojson.mesh(us, us.objects.states, function(a, b) { return a !== b; }))
          .attr("class", "states")
          .attr("d", path);
    }
    
})
</script>

In [5]:
from census_api_utils import (counties, census_labels, diversity, FINAL_LABELS)
r = list(counties(census_labels()))

In [6]:
from pandas import DataFrame

counties_df = DataFrame(r)
counties_df = diversity(counties_df)
counties_df[FINAL_LABELS].head()

,NAME,Total,White,Black,Asian,Hispanic,Other,p_White,p_Black,p_Asian,p_Hispanic,p_Other,entropy5,entropy4,entropy_rice,gini_simpson
0,Autauga County,54571,42154,9595,467,1310,1045,0.772462,0.175826,0.008558,0.024005,0.019149,0.441816,0.453294,0.458294,0.371372
1,Baldwin County,182265,152200,16966,1340,7992,3767,0.835048,0.093084,0.007352,0.043848,0.020668,0.388299,0.386196,0.392968,0.291627
2,Barbour County,27457,12837,12820,107,1387,306,0.467531,0.466912,0.003897,0.050515,0.011145,0.580086,0.636407,0.637309,0.560717
3,Bibb County,22915,17191,5024,22,406,272,0.750207,0.219245,0.000960,0.017718,0.011870,0.421943,0.448712,0.451897,0.388665
4,Blount County,57322,50952,724,115,4626,905,0.888873,0.012630,0.002006,0.080702,0.015788,0.274015,0.263741,0.270876,0.202978


In [7]:
counties_df['fips_for_map'] = counties_df.apply(lambda s: str(int(s['state']))+s['county'], axis=1)

In [8]:
df = DataFrame()
df[['id','diversity']] = counties_df[['fips_for_map', 'entropy5']]

In [9]:
df.to_csv('temp/entropy5.tsv', sep="\t", index=False)

In [10]:
!head temp/entropy5.tsv

id	diversity
1001	0.4418158563234724
1003	0.3882988721091073
1005	0.5800861120053159
1007	0.42194274556423633
1009	0.27401484623038297
1011	0.5101470309826847
1013	0.5120159003143947
1015	0.48169478052525105
1017	0.5139170933218179


In [11]:
%%html
<div id="diversity_map" style="height:600px; width:100%"></div>
<script>
// https://github.com/mbostock/d3/issues/1693
require.config({
  paths: {
    d3: "http://d3js.org/d3.v3.min",
    queue: "http://d3js.org/queue.v1.min",
    topojson: "http://d3js.org/topojson.v1.min"
  }
});

require(["d3", "queue", "topojson"], function(d3, queue, topojson) {
  console.log(d3.version);
  console.log(queue.version);
  console.log(topojson.version);


    var width = 960,
        height = 500;

    var rateById = d3.map();

    var quantize = d3.scale.quantize()
        .domain([0, 1.0])
        .range(d3.range(9).map(function(i) { return "q" + i + "-9"; }));

    var path = d3.geo.path();

    var svg = d3.select('#diversity_map').append("svg")
        .attr("width", width)
        .attr("height", height);

    queue()
        .defer(d3.json, "files/data/us.json")
        .defer(d3.tsv, "files/temp/entropy5.tsv", function(d) { rateById.set(d.id, +d.diversity); })
        .await(ready);

    function ready(error, us) {
      svg.append("g")
          .attr("class", "counties")
        .selectAll("path")
          .data(topojson.feature(us, us.objects.counties).features)
        .enter().append("path")
          .attr("class", function(d) { return quantize(rateById.get(d.id)); })
          .attr("d", path);

      svg.append("path")
          .datum(topojson.mesh(us, us.objects.states, function(a, b) { return a !== b; }))
          .attr("class", "states")
          .attr("d", path);
    }
    
})
</script>